# Network Analysis of Harry Potter Book Series

## Setup

In [3]:
!pip install -r requirements.txt
from sentiment import *

     |████████████████████████████████| 115 kB 24.5 MB/s 
     |████████████████████████████████| 125 kB 54.0 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


## Function Definitions for Text Analysis

`happiness(doc)` takes a list of words and computes the average happiness score using the Hedonometer

`emotion_score(doc)` takes a list of words and computes a dictionary of average emotion scores among the emotions: _Anger, Anticipation,	Disgust, Fear, Joy, Sadness, Surprise_ and _Trust_

`vader_sentiment(doc)` takes a list of words and/or sentences and computes the average VADER compund score

`TF_IDF(docs_to_analyse, all_docs)` computes TF and TF-IDF score of terms in docs_to_analyse. all_docs are used to comput IDF scores 


In [5]:
import pandas as pd
import pickle as pkl
from clean_books import clean_book

In [103]:
chapter_info = pd.read_pickle('data/chapter_dataframe.pkl')
# Add book number to chapters
idx = list(np.where(chapter_info["Local Chapter"].values == 1)[0])
for i in range(7):
    chapter_info.loc[idx[i]:(idx[i+1] if i+1 < len(idx) else None), "Book"] = int(i+1)

chapter_info

,Global Chapter,Local Chapter,Title,Approx Story Time,Book
0,1,1,The Boy Who Lived,1981-11-01,1.0
1,2,2,The Vanishing Glass,1991-06-23,1.0
2,3,3,The Letters from No One,1991-07-23,1.0
3,4,4,The Keeper of Keys,1991-07-31,1.0
4,5,5,Diagon Alley,1991-07-31,1.0
...,...,...,...,...,...
194,195,33,The Prince’s Tale,1998-05-02,7.0
195,196,34,The Forest Again,1998-05-02,7.0
196,197,35,King’s Cross,1998-05-02,7.0
197,198,36,The Flaw in the Plan,1998-05-02,7.0


In [8]:
with open('data/characters_by_house.pkl', 'rb') as file:
    characters_by_communities = pkl.load(file)

characters_by_communities

{'Gryffindor': ['Albus Dumbledore',
  'Alicia Spinnet',
  'Andrew Kirke',
  'Angelina Johnson',
  'Bill Weasley',
  'Celestina Warbeck',
  'Charlie Weasley',
  'Cormac McLaggen',
  'Dean Thomas',
  'Demelza Robins',
  'Dennis Creevey',
  'Euan Abercrombie',
  'Fred Weasley',
  'Geoffrey Hooper',
  'George Weasley',
  'Ginny Weasley',
  'Godric Gryffindor',
  'Harry Potter',
  'Jack Sloper',
  'James Potter',
  'James Sirius Potter',
  'Jimmy Peakes',
  'Katie Bell',
  'Kenneth Towler',
  'Lavender Brown',
  'Lee Jordan',
  'Lily Potter',
  'Minerva McGonagall',
  'Natalie McDonald',
  'Nearly-Headless Nick',
  'Neville Longbottom',
  'Oliver Wood',
  'Panju Weasley',
  'Parvati Patil',
  'Patricia Stimpson',
  'Percy Weasley',
  'Peter Pettigrew',
  'Remus Lupin',
  'Ritchie Coote',
  'Romilda Vane',
  'Ron Weasley',
  'Rose Granger-Weasley',
  'Rubeus Hagrid',
  'Seamus Finnigan',
  'Sir Cadogan',
  'Vicky Frobisher',
  'Wormtail',
  'Yann Fredericks'],
 'Hufflepuff': ['Artemisia Lufk

In [148]:
import os, re
path = "data/books/"
books = os.listdir(path)
books.sort()
for i in range(1):
    book = clean_book(path + books[i])
    chapters = [chapter.upper() for chapter in chapter_info.loc[chapter_info["Book"] == i+1, "Title"]]
    regexPattern = '|'.join(chapters)
    book_ = re.split(regexPattern, book)


In [155]:
print(chapters)

['THE BOY WHO LIVED', 'THE VANISHING GLASS', 'THE LETTERS FROM NO ONE', 'THE KEEPER OF KEYS', 'DIAGON ALLEY', 'THE JOURNEY FROM PLATFORM NINE AND THREE-QUARTERS', 'THE SORTING HAT', 'THE POTIONS MASTER', 'THE MIDNIGHT DUEL', "HALLOWE'EN", 'QUIDDITCH', 'THE MIRROR OF ERISED', 'NICOLAS FLAMEL', 'NORBERT THE NORWEGIAN RIDGEBACK', 'THE FORBIDDEN FOREST', 'THROUGH THE TRAPDOOR', 'THE MAN WITH TWO FACES']


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=85fb65b4-b596-4730-837e-04e86eafe419' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>